# Data grabbing notebook
This notebook contains the data grabbing code for the industry portfolios and the crypto returns. The main parameters for the two scripts are put here in the beginning for your convenience. 

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf

start_date = datetime(2017, 11, 9)
end_date = datetime(2023, 9, 30)

data_industry = '12_Industry_portfolios_Daily'
#data_industry = 
#data_industry = 
#data_industry = 

tickers_crypto = ["BTC-USD", "ETH-USD", "XRP-USD"] #add more tickers here
#tickers_crypto = ["BTC-USD", "ETH-USD", "XRP-USD", "BNB-USD"] 
#tickers_crypto = ["BTC-USD", "ETH-USD", "XRP-USD", "BNB-USD", "USDT-USD"]

### Industry portfolios from Ken French data library

In [2]:
# change this dummy variable to 0 for value weighted returns
equal_weight = 1 

# grabbing the data
industry_portfolios = web.DataReader(data_industry, 'famafrench', start=start_date, end = end_date)

# We grabbed a dictionary of lists, key 0,1,2 = (avg vw returns), (avg equal w returns), (DESCR) 
df = industry_portfolios[equal_weight]/100

df.to_csv(f'../../data/industry_returns.csv')

# this uses date_parser instead of date_format so is suboptimal speed wise. With our data size, not really a problem.
# But this approach will be depreciated in future versions. 


/tmp/ipykernel_49013/2284503902.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industry_portfolios = web.DataReader(data_industry, 'famafrench', start=start_date, end = end_date)
/tmp/ipykernel_49013/2284503902.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industry_portfolios = web.DataReader(data_industry, 'famafrench', start=start_date, end = end_date)


### Crypto returns from yahoo finance

In [3]:
# # empty df for storing values
# crypto_returns = pd.DataFrame()

# # loop for grabbing returns for provided tickers
# for ticker in tickers_crypto:
#     data = yf.download(ticker, start=start_date, end=end_date)
    
#     # computing log returns w adjusted closing prices
#     simple_return = data['Adj Close'].pct_change()
    
#     # converting to df and naming column
#     simple_return = simple_return.to_frame(name=ticker) 

#     if crypto_returns.empty: # handles first iteration
#         crypto_returns = simple_return 
#     else: # i.e. the subsequent iterations
#         crypto_returns = crypto_returns.join(simple_return, how="outer")

# crypto_returns = crypto_returns[1:] # removing first row (NaN)

# crypto_returns.to_csv('../../data/crypto_returns.csv')

### Crypto prices yahoo finance

In [4]:
# empty df for storing values
crypto_prices = pd.DataFrame()

# loop for grabbing returns for provided tickers
for ticker in tickers_crypto:
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # grabbing adj close price
    price = data['Adj Close']
    
    # converting to df and naming column
    price = price.to_frame(name=ticker) 

    if price.empty: # handles first iteration
        price = simple_return 
    else: # i.e. the subsequent iterations
        crypto_prices = crypto_prices.join(price, how="outer")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Since the crypto data trades on dates where the Industry stocks do not, we need to calculate the crypto returns between the dates we have return data for the industry. 

### Adjusting the start and end date to match both our datasets

In [5]:
# This shouldn't be needed with the approach we used.

start_date = max(df.index.min(), crypto_prices.index.min())
end_date = min(df.index.max(), crypto_prices.index.max())

print(start_date) 
print(end_date)

2017-11-09 00:00:00
2023-09-29 00:00:00


### Removing crypto price data for days that industry doesn't trade

This approach should underestimate the volatility of the crypto a bit maybe? It will definetly skew the reflection of the true return behavior of the cryptos a little bit. But this effect can be argued to be marginal. (we have some information loss since the removed trading days can include some significant price movements --> thus the variance could be under/over estimated)


In [6]:
# Reindex crypto_prices to only include dates that are present in df
aligned_crypto_prices = crypto_prices.reindex(df.index)

aligned_crypto_prices.dropna(inplace=True) # also removing industry returns on dates we don't have crypto returns
# the approach on above line of code is pretty flawed?

crypto_returns = aligned_crypto_prices.pct_change()
crypto_returns

,BTC-USD,ETH-USD,XRP-USD
Date,,,
2017-11-09,NaN,NaN,NaN
2017-11-10,-0.073554,-0.067411,-0.050600
2017-11-13,-0.008862,0.058355,-0.014728
2017-11-14,0.011626,0.066037,0.031375
2017-11-15,0.102444,-0.012659,0.014922
...,...,...,...
2023-09-25,-0.010575,-0.003104,-0.013754
2023-09-26,-0.003089,0.003207,-0.007425
2023-09-27,0.005167,0.002557,-0.004208


### Comparing the index values in the two df:s

In [7]:
# Convert the indices to sets
dates_in_df = set(df.index)
dates_in_crypto_returns = set(crypto_returns.index)

# Find dates that are only in crypto_returns
dates_only_in_crypto_returns = dates_in_crypto_returns - dates_in_df

# Convert to a sorted list for easier viewing or further processing
dates_only_in_crypto_returns = sorted(list(dates_only_in_crypto_returns))

# Print or inspect the dates
print(dates_only_in_crypto_returns)


[]


### Saving crypto returns as csv

In [8]:
crypto_returns = crypto_returns[1:] # removing first row (NaN)

crypto_returns.to_csv('../../data/crypto_returns.csv')